#Decision tree



In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/TP 2023/TP2/PARTE 3 : MODELS/TRAIN_FINISHED.parq')
test = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/TP 2023/TP2/PARTE 3 : MODELS/TEST_FINISHED.parq')
val = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/TP 2023/TP2/PARTE 3 : MODELS/VAL_FINISHED.parq')

In [3]:
X_train = train.loc[:, ~train.columns.isin(['label', "attacker_ip_enum", 'HAS_TCP?', 'HAS_UDP?', 'watcher_as_name'])]
Y_train = train.loc[:, 'label']
X_test = test.loc[:, ~test.columns.isin(['label', "attacker_ip_enum", 'HAS_TCP?', 'HAS_UDP?', 'watcher_as_name'])]
Y_test = test.loc[:, 'label']
X_val = val.loc[:, ~val.columns.isin(['label', "attacker_ip_enum", 'HAS_TCP?', 'HAS_UDP?', 'watcher_as_name'])]
Y_val = val.loc[:, 'label']
del train
del val
del test

In [4]:

model = DecisionTreeClassifier()

param_grid = {
    'criterion': ['entropy', "log_loss"],
    'splitter': ['best'],
    'max_depth': [None, 20, 40],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [ 'sqrt', 'log2'],
    'max_leaf_nodes': [None,  15, 30],
    'min_impurity_decrease': [0.0,  0.2, 0.4],
    'class_weight': ['balanced'],
    "random_state":[1996],
}


grid_search = GridSearchCV(model, param_grid, scoring='f1', cv=2, verbose=3, return_train_score = True)
grid_search.fit(X_train, Y_train, )


best_params = grid_search.best_params_


best_model = DecisionTreeClassifier(**best_params)
best_model.fit(X_train, Y_train)

y_val_pred = best_model.predict(X_val)
f1_val = f1_score(Y_val, y_val_pred)

y_test_pred = best_model.predict(X_test)
f1_test = f1_score(Y_test, y_test_pred)

print("Best Hyperparameters:", best_params)

print(f'Validation F1 Score: {f1_val}')

print(f'Test F1 Score: {f1_test}')

Fitting 2 folds for each of 648 candidates, totalling 1296 fits
[CV 1/2] END class_weight=balanced, criterion=entropy, max_depth=None, max_features=sqrt, max_leaf_nodes=None, min_impurity_decrease=0.0, min_samples_leaf=1, min_samples_split=2, random_state=1996, splitter=best;, score=(train=0.983, test=0.976) total time=  10.2s
[CV 2/2] END class_weight=balanced, criterion=entropy, max_depth=None, max_features=sqrt, max_leaf_nodes=None, min_impurity_decrease=0.0, min_samples_leaf=1, min_samples_split=2, random_state=1996, splitter=best;, score=(train=0.984, test=0.977) total time=  14.8s
[CV 1/2] END class_weight=balanced, criterion=entropy, max_depth=None, max_features=sqrt, max_leaf_nodes=None, min_impurity_decrease=0.0, min_samples_leaf=1, min_samples_split=5, random_state=1996, splitter=best;, score=(train=0.981, test=0.975) total time=   4.9s
[CV 2/2] END class_weight=balanced, criterion=entropy, max_depth=None, max_features=sqrt, max_leaf_nodes=None, min_impurity_decrease=0.0, min

Best Hyperparameters: {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'random_state': 1996, 'splitter': 'best'}
Validation F1 Score: 0.40358495795990024
Test F1 Score: 0.31412601747130825